# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 12:17PM,252995,10,Enova-11095,Emerginnova,Released
1,Dec 13 2022 12:17PM,252995,10,DNMD-28325,Emerginnova,Released
2,Dec 13 2022 12:17PM,252995,10,DNMD-28327,Emerginnova,Released
3,Dec 13 2022 12:17PM,252993,10,0086242928,ISDIN Corporation,Released
4,Dec 13 2022 12:17PM,252993,10,0086245905,ISDIN Corporation,Released
5,Dec 13 2022 12:17PM,252993,10,0086245906,ISDIN Corporation,Released
6,Dec 13 2022 12:17PM,252993,10,0086245940,ISDIN Corporation,Released
7,Dec 13 2022 12:06PM,252988,10,MSP216706,"Methapharm, Inc.",Released
8,Dec 13 2022 12:06PM,252988,10,MSP216707,"Methapharm, Inc.",Released
9,Dec 13 2022 12:06PM,252988,10,MSP216709,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,252987,Executing,1
41,252988,Released,19
42,252990,Released,1
43,252993,Released,4
44,252995,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252987,NaN,1.0,NaN
252988,NaN,NaN,19.0
252990,NaN,NaN,1.0
252993,NaN,NaN,4.0
252995,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0.0,0.0,1.0
252861,0.0,3.0,2.0
252891,1.0,0.0,4.0
252899,0.0,0.0,1.0
252926,0.0,0.0,12.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252848,0,0,1
252861,0,3,2
252891,1,0,4
252899,0,0,1
252926,0,0,12


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,0,0,1
1,252861,0,3,2
2,252891,1,0,4
3,252899,0,0,1
4,252926,0,0,12


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252848,,,1
1,252861,,3,2
2,252891,1,,4
3,252899,,,1
4,252926,,,12


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 12:17PM,252995,10,Emerginnova
3,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation
7,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc."
26,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC"
27,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated"
28,Dec 13 2022 11:48AM,252986,21,"NBTY Global, Inc."
29,Dec 13 2022 11:47AM,252985,21,"NBTY Global, Inc."
30,Dec 13 2022 11:45AM,252984,20,"Emersa Waterbox, LLC"
31,Dec 13 2022 11:36AM,252982,10,Eywa Pharma Inc.
32,Dec 13 2022 11:33AM,252981,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 12:17PM,252995,10,Emerginnova,,,3
1,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,,,4
2,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",,,19
3,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC",,,1
4,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated",,1,
5,Dec 13 2022 11:48AM,252986,21,"NBTY Global, Inc.",,,1
6,Dec 13 2022 11:47AM,252985,21,"NBTY Global, Inc.",,,1
7,Dec 13 2022 11:45AM,252984,20,"Emersa Waterbox, LLC",,,1
8,Dec 13 2022 11:36AM,252982,10,Eywa Pharma Inc.,,,1
9,Dec 13 2022 11:33AM,252981,10,ISDIN Corporation,,1,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:17PM,252995,10,Emerginnova,3,,
1,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,4,,
2,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",19,,
3,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC",1,,
4,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated",,1,
5,Dec 13 2022 11:48AM,252986,21,"NBTY Global, Inc.",1,,
6,Dec 13 2022 11:47AM,252985,21,"NBTY Global, Inc.",1,,
7,Dec 13 2022 11:45AM,252984,20,"Emersa Waterbox, LLC",1,,
8,Dec 13 2022 11:36AM,252982,10,Eywa Pharma Inc.,1,,
9,Dec 13 2022 11:33AM,252981,10,ISDIN Corporation,4,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:17PM,252995,10,Emerginnova,3,,
1,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,4,,
2,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",19,,
3,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC",1,,
4,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:17PM,252995,10,Emerginnova,3.0,NaN,NaN
1,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,4.0,NaN,NaN
2,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",19.0,NaN,NaN
3,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC",1.0,NaN,NaN
4,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 12:17PM,252995,10,Emerginnova,3.0,0.0,0.0
1,Dec 13 2022 12:17PM,252993,10,ISDIN Corporation,4.0,0.0,0.0
2,Dec 13 2022 12:06PM,252988,10,"Methapharm, Inc.",19.0,0.0,0.0
3,Dec 13 2022 12:06PM,252990,10,"Virtue Rx, LLC",1.0,0.0,0.0
4,Dec 13 2022 11:55AM,252987,102,"Senseonics, Incorporated",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5059165,173.0,10.0,1.0
102,252987,0.0,1.0,0.0
12,505931,4.0,0.0,0.0
15,758799,94.0,59.0,10.0
16,1011787,7.0,0.0,0.0
17,252964,1.0,0.0,0.0
19,252848,1.0,0.0,0.0
20,505947,2.0,0.0,0.0
21,505971,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5059165,173.0,10.0,1.0
1,102,252987,0.0,1.0,0.0
2,12,505931,4.0,0.0,0.0
3,15,758799,94.0,59.0,10.0
4,16,1011787,7.0,0.0,0.0
5,17,252964,1.0,0.0,0.0
6,19,252848,1.0,0.0,0.0
7,20,505947,2.0,0.0,0.0
8,21,505971,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,173.0,10.0,1.0
1,102,0.0,1.0,0.0
2,12,4.0,0.0,0.0
3,15,94.0,59.0,10.0
4,16,7.0,0.0,0.0
5,17,1.0,0.0,0.0
6,19,1.0,0.0,0.0
7,20,2.0,0.0,0.0
8,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,173.0
1,102,Released,0.0
2,12,Released,4.0
3,15,Released,94.0
4,16,Released,7.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21
Status,,,,,,,,,
Completed,1.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0
Released,173.0,0.0,4.0,94.0,7.0,1.0,1.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21
0,Completed,1.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0
2,Released,173.0,0.0,4.0,94.0,7.0,1.0,1.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21
0,Completed,1.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0
2,Released,173.0,0.0,4.0,94.0,7.0,1.0,1.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()